# Doc2Query

In [1]:
#from Doc2Query.Doc2Query import Doc2Query
from Doc2Query import Doc2Query
import pyterrier as pt
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
# PyTerrier dataset to pass
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
type(pt_dataset)

pyterrier.datasets.IRDSDataset

### Expand the Documents

In [4]:
# Create an instance of Doc2Query
Doc2Query = Doc2Query("google/flan-t5-small", temperatur=None)

In [5]:
# Generate queries for the dataset and extend the documents by the queries
expanded_documents = Doc2Query.expandDocumentsByQueries('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 55643.10it/s]
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
expanded_documents[['text']].to_dict()

{'text': {0: "A Study on Word Similarity using Context Vector Models\n\n\n There is a need to measure word similarity when processing natural languages, especially when using generalization, classification, or example -based approaches. Usually, measures of similarity between two words are defined according to the distance between their semantic classes in a semantic taxonomy . The taxonomy approaches are more or less semantic -based that do not consider syntactic similarit ies. However, in real applications, both semantic and syntactic similarities are required and weighted differently. Word similarity based on context vectors is a mixture of syntactic and semantic similarit ies. In this paper, we propose using only syntactic related co-occurrences as context vectors and adopt information theoretic models to solve the problems of data sparseness and characteristic precision. The probabilistic distribution of co-occurrence context features is derived by parsing the contextual environme

In [ ]:
# Create the index using PyTerrier
indexer = pt.IterDictIndexer(
    "./indexes/index_Doc2Query-flan-t5-small",
    overwrite=True,
    fields=["text"],
    meta=["docno"]
)
# Index the documents
indexref = indexer.index(expanded_documents)

# Retrieve documents using BM25
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")

# Perform retrieval
#queries_df = pt_dataset.get_topics()
#run = bm25.transform(queries_df)
run = bm25(pt_dataset.get_topics('text'))

# Evaluate the results
qrels_df = pt_dataset.get_qrels()
eval = pt.Evaluate(run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)

In [ ]:
# Filter run to include only judged documents
qrels_df = pt_dataset.get_qrels()
judged_docnos = qrels_df['docno'].unique()
filtered_run = run[run['docno'].isin(judged_docnos)]

# Evaluate the results
eval = pt.Evaluate(filtered_run, qrels_df, metrics=["map", "ndcg", "ndcg_cut.10", "recip_rank", "recall_100"])
print("Evaluation Metrics:")
print(eval)

In [ ]:
# Persist the run file for subsequent evaluations
#persist_and_normalize_run(run, system_name='Doc2Query-flan-t5-small-BM25-judged_only', default_output='../runs')